In [1]:
!pip install openai==0.28 requests


  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-win_amd64.whl.metadata (7.7 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp312-cp312-win_amd64.whl.metadata (32 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
Using cached aiohttp-3.9.5-cp312-cp312-win_amd64.whl (369 kB)
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Using cached frozenlist-1.4.1-cp312-cp312-win_amd64.whl (50 kB)
Using cached multidict-6.0.5-cp312-cp312-win_amd64.whl (27 kB)
Using cached yarl-1.9.4-cp312-cp312-win_amd64.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.35.15
    Uninstalling openai-1.35.15:
      Successfully uninstalled openai-1.35.15


In [2]:
!pip install gradio

In [3]:
import gradio as gr

c:\project folder\Weather_Assistant\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import openai

In [5]:
# Initialize OpenAI client
openai.api_key = "OPENAI_API_KEY"

In [7]:
import requests

def get_current_weather(location, unit='celsius'):
    api_key = "WEATHER_API_KEY"
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
    response = requests.get(base_url)
    data = response.json()

    weather_description = data['weather'][0]['description']
    return {
        "location": location,
        "temperature": data['main']['temp'],
        "weather": weather_description
    }

In [8]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, e.g. San Francisco"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [12]:
def weather_chat(user_message):
    messages=[]
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": "You are a weather bot. Answer the questions related to weather only. Else politely say no to the user"})
        # Send the messages to OpenAI
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",

            messages=messages,
            functions=functions
        )


    try:
        function_call = response['choices'][0]['message']['function_call']
        arguments = eval(function_call['arguments'])

        # Fetch weather data using the extracted arguments
        weather_data = get_current_weather(arguments['location'])

        # Append the function call and weather data to the messages
        messages.append({"role": "assistant", "content": None, "function_call": {"name": "get_current_weather", "arguments": str(arguments)}})
        messages.append({"role": "function", "name": "get_current_weather", "content": str(weather_data)})

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        return response['choices'][0]['message']['content']
    except Exception as e:
        return "I'm here to provide weather updates. Please ask me questions related to weather."



In [13]:
# Define Gradio interface
iface = gr.Interface(
    fn=weather_chat,
    inputs=gr.Textbox(label="Weather Queries"),
    outputs=gr.Textbox(label="Weather Updates"),
    title = "Weather Bot",
    description = "Ask me anything about weather!"
)

# Launch the Gradio interface
iface.launch(share="True")

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://b2e81eb943118ffe44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
